<a href="https://colab.research.google.com/github/SimperPhi/Urban_Sound_Classification_Project/blob/master/UrbanSound8KCNNClassfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libfluidsynth1

In [ ]:
!pip install resampy

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from IPython.display import Audio

# Ses dosyasının tam yolu
ses_dosya_yolu = "/content/drive/MyDrive/02-Urban8k-grup6/audio/fold1/101415-3-0-2.wav"

# Ses dosyasını oynat
Audio(ses_dosya_yolu)


In [ ]:
import librosa

librosa_file_path = "/content/drive/MyDrive/02-Urban8k-grup6/audio/fold1/101415-3-0-2.wav"
librosa_audio_data, librosa_sample_rate = librosa.load(librosa_file_path)


In [ ]:
print (librosa_audio_data)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(librosa_audio_data)
plt.show()

In [ ]:
mfccs=librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)

In [ ]:
mfccs

In [ ]:
from zmq import METADATA
audio_dataset_path="/content/drive/MyDrive/02-Urban8k-grup6/audio"
metadata=pd.read_csv("/content/drive/MyDrive/02-Urban8k-grup6/metadata/UrbanSound8K.csv")
metadata.head()

In [ ]:
### Check whether the dataset is imbalanced
metadata['class'].value_counts()

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features


In [ ]:
!pip install lazy_loader

In [ ]:
!pip install resampy
import numpy as np
from tqdm import tqdm

extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

In [ ]:
extracted_features_df.tail()

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape

In [ ]:
y

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X_train

In [ ]:
y

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:

model=Sequential()
###first layer
model.add(Dense(125,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 300
num_batch_size = 32

In [ ]:
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
#Buradan itibaren çalıştırma

In [ ]:
folder_path = "/content/drive/MyDrive/02-Urban8k-grup6/audio/fold2/"

# Klasördeki tüm .wav dosyalarını alın
wav_files = glob.glob(os.path.join(folder_path, "*.wav"))

# Rastgele bir ses dosyası seçin
random_wav_file = random.choice(wav_files)

# Seçilen dosyayı yükleyin
audio, sample_rate = librosa.load(random_wav_file, res_type='kaiser_fast')
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
print(f"Seçilen Dosya: {random_wav_file}")


In [ ]:
X_test[1]

In [ ]:
mfccs_scaled_features

In [ ]:
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

In [ ]:
mfccs_scaled_features.shape

In [ ]:
result_array=model.predict(mfccs_scaled_features)

In [ ]:
result_array

In [ ]:
result_classes=["klima sesi","araba korna sesi","çocuk sesleri","köpek havlaması",
"sonaj","motor sesi","silah sesi","darbeli kırıcı","siren","sokakta müzik"]

In [ ]:
result_classes

In [ ]:
result=np.argmax(result_array[0])

In [ ]:
result_classes[result]

In [ ]:
if result_classes[result]=='köpek havlaması' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/dog.jpeg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='klima sesi' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/klima.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='araba korna sesi' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/araba.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='çocuk sesleri' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/çocuk sesleri.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='sonaj' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/sonaj.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='motor sesi' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/motor sesi.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='silah sesi' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/silah sesi.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='darbeli kırıcı' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/darbeli.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='siren' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/siren.jpg')
  imgplot=plt.imshow(img)
  plt.show()
if result_classes[result]=='sokakta müzik' :
  import matplotlib.image as mpimg
  plt.figure(figsize=(12,4))
  img = mpimg.imread('/content/drive/MyDrive/müzik.jpg')
  imgplot=plt.imshow(img)
  plt.show()

In [ ]:
display(Audio(data=audio, rate=sample_rate))

In [ ]:
librosa.display.waveshow(audio, sr=sample_rate)